# <center>Neural network with Pytorch</center>

In this notebook we use `torch.module` and `torch.optimizer` to create our neural network. We train it on MNIST. 

In [1]:
from matplotlib import pyplot
import math
import numpy as np
import torch
import torchvision.datasets as datasets
import torch.nn.functional as F
from torch import nn
from torch import optim

In [2]:
loss_func = F.cross_entropy

def accuracy(Y_hat, Y):
    preds = torch.argmax(Y_hat, dim=1)
    return (preds == Y).float().mean()

In [3]:
#import data
mnist_trainset = datasets.MNIST(root='../data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='../data', train=False, download=True, transform=None)

#load trainset into tensors
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=1, shuffle=True)
X_train = train_loader.dataset.data
Y_train = train_loader.dataset.targets

#load testset into tensors
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10000, shuffle=False)
X_test = test_loader.dataset.data
Y_test = test_loader.dataset.targets

In [4]:
#scale data to [0:1] and convert to float32
X_train = (X_train.to(dtype=torch.float32) / X_train.max().to(dtype=torch.float32))
X_test = (X_test.to(dtype=torch.float32) / X_test.max().to(dtype=torch.float32))

#Flatten train and test data
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)

print("Train examples : ",X_train.shape[0])
print("Test examples : ",X_test.shape[0])
print("Nb of features : ",X_train.shape[1])

Train examples :  60000
Test examples :  10000
Nb of features :  784


In [5]:
class NeuralNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784,512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512,256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256,128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128,64)  
        self.bn4 = nn.BatchNorm1d(64)
        self.fc5 = nn.Linear(64,32)
        self.bn5 = nn.BatchNorm1d(32)
        self.fc6 = nn.Linear(32,16)   
        self.bn6 = nn.BatchNorm1d(16)
        self.fc7 = nn.Linear(16,10)        
 
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.bn1(x)
        x = torch.relu(self.fc2(x))
        x = self.bn2(x)
        x = torch.relu(self.fc3(x))
        x = self.bn3(x)
        x = torch.relu(self.fc4(x))
        x = self.bn4(x)
        x = torch.relu(self.fc5(x))
        x = self.bn5(x)
        x = torch.relu(self.fc6(x))
        x = self.bn6(x)
        x = torch.sigmoid(self.fc7(x))
        return x
                            
                            
    def fit(self,optimizer,epochs,batch_size):
        n = X_train.shape[0]
        for epoch in range(epochs):
            for i in range((n - 1) // batch_size + 1):
                
                X = X_train[ i * batch_size : (i+1) * batch_size ]
                Y = Y_train[ i * batch_size : (i+1) * batch_size ]
                
                pred = self.forward( X )
                loss = loss_func( pred , Y )

                loss.backward()
                opt.step()
                opt.zero_grad()
            print(epoch+1,"\t",loss.item())

In [6]:
layers = [784,512,256,128,64,32,16,10]

model = NeuralNet()
opt = optim.SGD(model.parameters(), lr=1)
opt_2 = optim.Adam(model.parameters(), lr=0.1, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [76]:
model.fit(opt,20,600)

1 	 1.5673637390136719
2 	 1.5133841037750244
3 	 1.4983447790145874
4 	 1.4907454252243042
5 	 1.4882878065109253
6 	 1.4871314764022827
7 	 1.4854809045791626
8 	 1.484190583229065
9 	 1.485247015953064
10 	 1.4839437007904053
11 	 1.4839590787887573
12 	 1.482917308807373
13 	 1.4832372665405273
14 	 1.4835891723632812
15 	 1.482880711555481
16 	 1.4863377809524536
17 	 1.482969045639038
18 	 1.4832115173339844
19 	 1.4830360412597656
20 	 1.4823493957519531


In [77]:
preds = model.forward(X_test)


print("Accuracy on test set : ", round(accuracy( preds , Y_test).item(),3))

Accuracy on test set :  0.896


In [ ]:
model.train()